In [1]:
import numpy as np
import pandas as pd
from astropy.table import Table, vstack
from matplotlib import pyplot as plt
from glob import glob
from astropy.stats import LombScargle
from scipy import stats
from scipy.optimize import curve_fit
import warnings
from TESStools import *
import celerite
from celerite import terms
from scipy.optimize import minimize

In [2]:
paper_tics = [404850274,389437365,389565293,40404470,29984014,279957111,179305185]

In [3]:
for i, tid in enumerate(paper_tics):
    
    fname = np.unique(massive['CommonName'][massive['ticid']==tid])[0].replace('*','').replace('V','')
    lc, lc_smooth = lc_extract(get_lc_from_id(tid), smooth=128)
    
    if (fname == 'HD 268687'):
        lc, p = polynorm(lc, deg=7)
        time, flux, err = lc['Time'], lc['NormFlux'], lc['NormErr']
    else:
        time, flux, err = lc['Time'], lc['Flux'], lc['Err']
    
    good_fs, good_amps, good_phases = prewhiten(time, flux, err)

f_Ny = 323.89171072083246, f_R = 0.01778794028727421
10
Found 14 frequencies


NameError: name 'freqs' is not defined